<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Python Fundamentals: TITLE</h1>  

©2000-20 Chris Seddon 
</div>

## 1
Execute the following cell to activate styling for this tutorial

In [1]:
from IPython.display import HTML
HTML(f"<style>{open('my.css').read()}</style>")

## 2
In this tutorial, we will be showing how to wrap up C and C++ code into a module that can be called as if it were normal Python code; this is achieved by compiling C and C++ code into a shared library.

However, before we start this tutorial, I need to point out that Jupyter notebook doesn't always show the output of Python code.  This is because the notebook server runs code in a subshell and redirects this output of the shell to the browser.  But, some Python code runs in a subshell of a subshell and this output does not get redirected to the browser by the notebook server.

This is particularly true of Python "subprocess" module.  For example is we run code in the following cell, all we see is the reurn code from "subprocess.run()":

In [2]:
import subprocess
subprocess.run("ls -l".split())

CompletedProcess(args=['ls', '-l'], returncode=0)

## 3
We can circumvent this problem by using pipes.  The "subprocess" module can use pipes to get output from the subshell of the subshell as follows:

In [3]:
import subprocess
result = subprocess.run("ls -l".split(), stdout=subprocess.PIPE)
print (f"return code: {result.returncode} \nstdout = \n{result.stdout.decode()}")

return code: 0 
stdout = 
total 2424
-rw-r--r--   1 seddon  staff   10075 18 Mar 21:41 A02_mutability.ipynb
-rw-r--r--   1 seddon  staff    8696 18 Mar 21:39 A03_conditionals_and_loops.ipynb
-rw-r--r--   1 seddon  staff   15380 18 Mar 21:36 A04_lists_and_tuples.ipynb
-rw-r--r--   1 seddon  staff   20134 18 Mar 21:29 A05_functions.ipynb
-rw-r--r--   1 seddon  staff   13438 18 Mar 21:19 A06_dictionaries.ipynb
-rw-r--r--   1 seddon  staff   16867 18 Mar 21:16 A07_modules.ipynb
-rw-r--r--   1 seddon  staff   21106 18 Mar 21:14 A08_classes.ipynb
-rw-r--r--   1 seddon  staff   13205 18 Mar 21:12 A09_file_IO.ipynb
-rw-r--r--   1 seddon  staff   17004 17 Mar 19:45 A10_exception_handling.ipynb
-rw-r--r--   1 seddon  staff   11850 18 Mar 20:49 A11_comprehensions.ipynb
-rw-r--r--   1 seddon  staff   18419 18 Mar 20:51 A12_iterators_and_generators.ipynb
-rw-r--r--   1 seddon  staff   20932 16 Mar 23:44 A13_decorators.ipynb
-rw-r--r--   1 seddon  staff   21009 19 Mar 23:37 B01_numpy.ipynb
-rw-r--r-

## 4
For the rest of this tutorial, we will wrap up this functionality in a "run" function:

In [4]:
def run(cmd):
    result = subprocess.run (cmd.split(), stdout=subprocess.PIPE)
    print (f"return code = {result.returncode} \n{result.stdout.decode()}")
print("run command defined")

run command defined


## 5
Now we are ready to start the tutorial.

When we want to wrap up C and C++ code in a shared library we need to provide:
* the C/C++ code to go in the shared library
* setup.py: instructions on building the library

As an example of C code, we will write a Fibonacci module.  Here is the C code:

In [5]:
%%bash
cat resources/fibmodule.c

#include <Python.h>

int _fib(int n)
{
    if (n < 2)
        return n;
    else
        return _fib(n-1) + _fib(n-2);
}

static PyObject* fib(PyObject* self, PyObject* args)
{
    int n;

    if (!PyArg_ParseTuple(args, "i", &n))
        return NULL;

    return Py_BuildValue("i", _fib(n));
}

// method table
static PyMethodDef FibMethods[] = {
    {"fib", (PyCFunction)fib, METH_VARARGS, "Calculate the Fibonacci numbers."},
    {NULL, NULL, 0, NULL}
};

// module definition structure
static struct PyModuleDef moduleDefinition =
{
    PyModuleDef_HEAD_INIT,
    "fibonacci", /* name of module */
    "",          /* module documentation, may be NULL */
    -1,          /* size of per-interpreter state of the module, or -1 if the module keeps state in global variables. */
    FibMethods
};

// module initialization function
PyMODINIT_FUNC PyInit_fibonacci(void)
{
    return PyModule_Create(&moduleDefinition);
}



## 7
The C code is in 4 parts:

* the function<pre>int _fib(int n)</pre>
* binding code between C and Python in the function<pre>static PyObject* fib(PyObject* self, PyObject* args)</pre>
* interface definitions<pre>
method table
module definition structure</pre>
* Python entry point to the shared libary<pre>PyMODINIT_FUNC PyInit_fibonacci(void)</pre>  

The `setup.py` file is shown below:

In [6]:
%%bash
cat resources/setup.py

import sys
from distutils.core import setup, Extension
 
mymodule = Extension('fibonacci', sources = ['fibmodule.c'])

setup (name = 'FibonacciPackage',
        version = '1.0',
        author = "CRS Enterprises Ltd",
        author_email='seddon-software@keme.co.uk',
        maintainer = "CRS Enterprises Ltd",
        maintainer_email ='seddon-software@keme.co.uk',
        url='http://www.keme.co.uk/~seddon-software',
        description = "simple example",
        long_description = """This is a simple example,
            to wrap up a C module""",
        download_url = "local",
        classifiers = ["Developers", "Python Programming Course"],
        platforms = sys.platform,
        license = "none",
        ext_modules = [mymodule])


## 8
The key parts of the above file are:

* creating a "distutils.core.setup" object, containing build details <pre>setup(...)</pre>

* an Extension object defining which C files are to be compiled:
<pre>mymodule = Extension('fibonacci', sources = ['fibmodule.c'])</pre>

* the link between the "setup" object and the "Extension" object
<pre>ext_modules = [mymodule]</pre>

Python has a special way of building the shared library:

In [7]:
import subprocess,os,sys

pythonPath = os.path.dirname(sys.executable)
swigPath = "/usr/local/bin"
compilerPath = "/opt/local/bin"
#os.environ["PATH"] = pythonPath + os.pathsep + os.environ["PATH"]
os.environ["PATH"] = pythonPath + os.pathsep + swigPath + os.pathsep + compilerPath + os.pathsep + os.environ["PATH"]

os.chdir("/Users/seddon/home/workspace/Python3/Python3/notebooks/resources")

run("python setup.py -v build_ext")
run("ls -lR build")

return code = 0 
running build_ext
building 'fibonacci' extension
creating build
creating build/temp.macosx-10.7-x86_64-3.7
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/seddon/Anaconda37/anaconda3/include -arch x86_64 -I/Users/seddon/Anaconda37/anaconda3/include -arch x86_64 -I/Users/seddon/Anaconda37/anaconda3/include/python3.7m -c fibmodule.c -o build/temp.macosx-10.7-x86_64-3.7/fibmodule.o
creating build/lib.macosx-10.7-x86_64-3.7
gcc -bundle -undefined dynamic_lookup -L/Users/seddon/Anaconda37/anaconda3/lib -arch x86_64 -L/Users/seddon/Anaconda37/anaconda3/lib -arch x86_64 -arch x86_64 build/temp.macosx-10.7-x86_64-3.7/fibmodule.o -o build/lib.macosx-10.7-x86_64-3.7/fibonacci.cpython-37m-darwin.so

return code = 0 
total 0
drwxr-xr-x  3 seddon  staff  96 23 Mar 17:33 lib.macosx-10.7-x86_64-3.7
drwxr-xr-x  3 seddon  staff  96 23 Mar 17:33 temp.macosx-10.7-x86_64-3.7

build/lib.macosx-10.7-x86_64-3.7:
total 24
-rw

## 9
Python creates a shared object file with a ".so" extension in the build directory a some other temporary files.

We now need to install this shared object .  Since we do not have permission to install in Diamond's Anacoda distribution we will have to install locally in our home directory:

In [8]:
run("python setup.py install --record files.txt --user")

return code = 0 
running install
running build
running build_ext
running install_lib
copying build/lib.macosx-10.7-x86_64-3.7/fibonacci.cpython-37m-darwin.so -> /Users/seddon/.local/lib/python3.7/site-packages
running install_egg_info
Writing /Users/seddon/.local/lib/python3.7/site-packages/FibonacciPackage-1.0-py3.7.egg-info
writing list of installed files to 'files.txt'



## 10
Now we've installed our library, we should clean the build area:

In [9]:
run("python setup.py clean --all")
print("staging area cleaned")

return code = 0 
running clean
removing 'build/temp.macosx-10.7-x86_64-3.7' (and everything under it)
removing 'build/lib.macosx-10.7-x86_64-3.7' (and everything under it)
removing 'build'

staging area cleaned


## 11
Time to test our module:

In [10]:
import fibonacci

print((fibonacci.fib(10)))
print((fibonacci.fib(11)))
print((fibonacci.fib(12)))

55
89
144


## 12
So our module is working!

Note, we've been keeping a record of the files we've been creating to facilitate the "uninstall":

In [11]:
%%bash
cat files.txt

/Users/seddon/.local/lib/python3.7/site-packages/fibonacci.cpython-37m-darwin.so
/Users/seddon/.local/lib/python3.7/site-packages/FibonacciPackage-1.0-py3.7.egg-info


## 13
The following code will print out the "egg-info" file:

In [12]:
installedFiles = open("files.txt", "r")
for file in installedFiles:
    file = file.rstrip()
    if file.endswith("egg-info"):
        f = open(file, 'r')
        print(f.read())

Metadata-Version: 1.1
Name: FibonacciPackage
Version: 1.0
Summary: simple example
Home-page: http://www.keme.co.uk/~seddon-software
Author: CRS Enterprises Ltd
Author-email: seddon-software@keme.co.uk
License: none
Download-URL: local
Description: This is a simple example,
                    to wrap up a C module
Platform: darwin
Classifier: Developers
Classifier: Python Programming Course



## 14
Now we can unistall our library:

In [13]:
try:
    installedFiles = open("files.txt", "r")
    for file in installedFiles:
        print(f"rm {file}")
        run(f"rm {file}")

    print("rm files.txt")
    run("rm files.txt")
    print("Example uninstalled")
except:
    print("Example already uninstalled")

rm /Users/seddon/.local/lib/python3.7/site-packages/fibonacci.cpython-37m-darwin.so

return code = 0 

rm /Users/seddon/.local/lib/python3.7/site-packages/FibonacciPackage-1.0-py3.7.egg-info

return code = 0 

rm files.txt
return code = 0 

Example uninstalled
